In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.constraints import Constraint
from tensorflow.keras import backend

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from numpy import mean
from numpy import zeros
from numpy import ones
from numpy import hstack
from numpy.random import rand
from numpy.random import randn

In [ ]:
filename="DB_modified.csv"
dataset = pd.read_csv(filename, header=None)
dataset =np.array(dataset)[1:-1]

dataset=np.asfarray(dataset,float)
# dataset.shape

In [ ]:
X_original=dataset[:,:7]
X = X_original
# print(X)
# X=(X-np.mean(X))/np.std(X)

# X=dataset[:,2]
X[:,1]=(X[:,1]-np.mean(X[:,1]))/np.std(X[:,1])
X[:,2]=(X[:,2]-np.mean(X[:,2]))/np.std(X[:,2])
X[:,3]=(X[:,3]-np.mean(X[:,3]))/np.std(X[:,3])

X[:,4]=(X[:,4]-np.mean(X[:,4]))/np.std(X[:,4])
X[:,5]=(X[:,5]-np.mean(X[:,5]))/np.std(X[:,5])
X[:,6]=(X[:,6]-np.mean(X[:,6]))/np.std(X[:,6])
X[:,0]=(X[:,0]-np.mean(X[:,0]))/np.std(X[:,0])
# print(X)
y= dataset[:,7]
# print(X.shape, y.shape)

In [ ]:
class ClipConstraint(Constraint):
    # set clip value when initialized
    def __init__(self, clip_value):
        self.clip_value = clip_value
 
    # clip model weights to hypercube
    def __call__(self, weights):
        return backend.clip(weights, -self.clip_value, self.clip_value)
 
    # get the config
    def get_config(self):
        return {'clip_value': self.clip_value}

In [ ]:
def wasserstein_loss(y_true, y_pred):
    return backend.mean(y_true * y_pred)

In [ ]:
def define_discriminator(n_inputs=7):
    const = ClipConstraint(0.01)
    model = Sequential()
    model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='linear'))
    # compile model
    model.compile(loss=wasserstein_loss, optimizer='RMSprop', metrics=['accuracy'])
    return model

In [ ]:
def define_generator(latent_dim, n_outputs=7):
    model = Sequential()
    model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='tanh', kernel_initializer='he_uniform'))
    model.add(Dense(n_outputs, activation='linear'))
    return model

In [ ]:
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss=wasserstein_loss, optimizer='RMSprop')
    return model

In [ ]:
def generate_real_samples(n):
    X_return = X[np.random.randint(0, X.shape[0], size=n)]
    y = -ones((n, 1))
    return X_return, y


In [ ]:
def generate_latent_points(latent_dim, n):
    x_input = np.random.normal(0, 1,size = (latent_dim * n))
    # generate points in the latent space
#     x_input = randn(latent_dim * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim)
    return x_input


In [ ]:
def generate_fake_samples(generator, latent_dim, n):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = ones((n, 1))
    return X, y


In [ ]:
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    # prepare real samples
    x_real, y_real = generate_real_samples(n)
    # evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
    # evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print(epoch, acc_real, acc_fake)
    # scatter plot real and fake data points
    pyplot.scatter(x_real[:, 3], x_real[:, 6], color='red')
    pyplot.scatter(x_fake[:, 3], x_fake[:,6], color='blue')
    pyplot.show()

In [ ]:
def train(g_model, d_model, gan_model, latent_dim, n_epochs=4000, n_batch=64, n_eval=50, n_critic=5):
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)
    # lists for keeping track of loss
    c1_hist, c2_hist, g_hist = list(), list(), list()
    # manually enumerate epochs
    for i in range(n_epochs):
        # update the critic more than the generator
        c1_tmp, c2_tmp = list(), list()
        for _ in range(n_critic):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(half_batch)
            # update critic model weights
            c_loss1 = d_model.train_on_batch(X_real, y_real)
            c1_tmp.append(c_loss1)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update critic model weights
            c_loss2 = d_model.train_on_batch(X_fake, y_fake)
            c2_tmp.append(c_loss2)  
        c1_hist.append(mean(c1_tmp))
        c2_hist.append(mean(c2_tmp))
        # prepare points in latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = -ones((n_batch, 1))
        # update the generator via the critic's error
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        g_hist.append(g_loss)
#       # evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))
            g_model.save('./model/WGAN/100%DPs/wgan_'+str(i)+'.h5')
            summarize_performance(i, g_model, d_model, latent_dim)

In [ ]:
latent_dim = 10
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)